In [24]:
import os 
from tqdm import tqdm, trange
import torch 
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

In [25]:
data_path = "./data/names.txt"

with open(data_path, "r") as f:
    raw_names = f.readlines()

names_processed  = [name.lower().strip() for name in raw_names]

In [31]:
def get_vocab(list_of_names):
    vocab = {}
    idx = 0
    for each in list_of_names:
        for char in each:
            if char not in vocab:
                vocab[char] = idx
                idx+=1
    vocab["*"] = idx
    idx+=1
    vocab["<EOF>"] = idx
    return vocab

In [32]:
def pad(list_of_names):
    max_len = max([len(name) for name in list_of_names])
    out = []

    for name in list_of_names:
        req = max_len - len(name)
        chars = list(name)
        chars += ["<EOF>"] + ["*"] * req
        out.append(chars)
    return out

In [39]:
def list_to_torch(list_of_names, vocab):
    out = []
    for name in list_of_names:
        chars = [vocab[char] for char in name]
        out.append(chars)
    
    return torch.Tensor((out))
    


In [40]:
vocab = get_vocab(names_processed)
# print(vocab)
padded_names = pad(names_processed)
input_data = list_to_torch(padded_names, vocab)
print(input_data.shape)

torch.Size([18239, 16])


In [ ]:
class Gen(nn.Module):
    def __init__(self, num_layers, vocab):
        self.num_layers = num_layers
        self.vocab = vocab
        self.emb = nn.Embedding(128, len(self.vocab), padding_idx=self.vocab["*"])

        self.gru = nn.GRU()

    